<a href="https://colab.research.google.com/github/AntikGhalt/predicting_bankruptcy_AIDA/blob/main/AIDA_2_TEST2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### COLAB DRIVE

---



In [2]:
#@title RESET AND FILES NAME
%reset -f
#path data export
model_folder = "model_test/"

# list your files in the exact order you want their rows to appear
file_names = [
    "Aida_Export_TESTin_0_1000.xlsx",
    "Aida_Export_TESTout_100001_101000.xlsx",
]

test_files = [
    "Aida_Export_testD_001_200.xlsx",
    "Aida_Export_testD_201_400.xlsx",
    "Aida_Export_testD_401_600.xlsx",
    "Aida_Export_testD_601_800.xlsx",
    "Aida_Export_testD_801_1000.xlsx",
    "Aida_Export_testD_100001_100100.xlsx",
    "Aida_Export_testD_100101_100200.xlsx",
    "Aida_Export_testD_100201_100400.xlsx",
    "Aida_Export_testD_100401_100600.xlsx",
    "Aida_Export_testD_100601_100800.xlsx",
    "Aida_Export_testD_100801_101000.xlsx"
]

In [3]:
path_data = "/content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/" + model_folder
path_MODEL = "/content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/output/" + model_folder

In [4]:
from IPython.display import clear_output
!pip install pandas
#!pip install scikit-learn
#!pip install optuna

import pandas as pd
import os
import numpy as np
import re

"""
def safe_literal_eval(x):
    # If x is a list or numpy array and empty, return an empty list
    if isinstance(x, (list, np.ndarray)) and len(x) == 0:
        return []
    # For scalar values, check if x is NaN
    if not isinstance(x, (list, np.ndarray)) and pd.isna(x):
        return []

    # Otherwise, convert to string and strip any whitespace
    x = str(x).strip()

    try:
        return ast.literal_eval(x)
    except Exception:
        return x
"""
clear_output()

### DATA MANIPULATION

---




In [5]:
#@title FILES ≥ 15 (with per–chunk date/year fixing)

# 1) Regex for “year” or “date” at end of column name
date_pattern = re.compile(r'(year|date)$', re.IGNORECASE)

df_list = []
for fn in file_names:               # assumes file_names & path_data exist
    print(f"Processing file: {fn}")
    tmp = pd.read_excel(
        path_data + fn,
        sheet_name="Results",
        index_col=None,
        na_values=["n.a."],       # if you still have literal "n.a."
        keep_default_na=True
    )
    # 2) Normalize your key
    tmp['Tax code number'] = tmp['Tax code number'].astype('Int64')

    # 3) Find & fix every date/year column in this chunk
    for col in tmp.columns:
        if not date_pattern.search(col):
            continue

        ser = tmp[col]
        # a) Excel‐serial branch (for floats/ints)
        ser_num = pd.to_numeric(ser, errors='coerce')
        dt = pd.to_datetime(
            ser_num,
            unit='D',
            origin='1899-12-30',
            errors='coerce'
        )

        # b) String‐date fallback
        mask = dt.isna() & ser.notna()
        if mask.any():
            dt.loc[mask] = pd.to_datetime(
                ser[mask],
                dayfirst=True,
                errors='coerce'
            )

        tmp[col] = dt
        # debug print
        total = ser.notna().sum()
        good  = dt.notna().sum()
        print(f"  {col}: parsed {good}/{total} values into dates")

    df_list.append(tmp)

# 4) Concatenate end‐to‐end
df_start = pd.concat(df_list, ignore_index=True)
print("Final shape:", df_start.shape)


#df.iloc[46847:46857]
#df.tail(1300)
df_start.shape
#df_start.iloc[3000:3050]
#df_start

Processing file: Aida_Export_TESTin_0_1000.xlsx
  Incorporation year: parsed 998/998 values into dates
  Last accounting closing date: parsed 1000/1000 values into dates
Processing file: Aida_Export_TESTout_100001_101000.xlsx
  Incorporation year: parsed 1000/1000 values into dates
  Last accounting closing date: parsed 1000/1000 values into dates
Final shape: (3273, 16)


(3273, 16)

In [6]:
#@title Events Manipulation

# 2) Build your 'master' enterprises (one row per tax code)
master = (
    df_start[df_start['Tax code number'].notna()]
      .drop_duplicates(subset='Tax code number', keep='first')
)

# 3) Prepare the events table
events = (
    df_start[['Tax code number',
        'Procedure/cessazione',
        'Date of open procedure/cessazione',
        'Date of closure procedure']]
    .copy()
)
# fill down the tax‐code so every event row has its enterprise key
events['Tax code number'] = events['Tax code number'].ffill()

# keep only real events
events = events[events['Procedure/cessazione'].notna()].rename(columns={
    'Procedure/cessazione':        'Procedure',
    'Date of open procedure/cessazione': 'open_date',
    'Date of closure procedure':   'close_date'
})

# parse dates
events['open_date']  = pd.to_datetime(events['open_date'],  dayfirst=True, errors='coerce')
events['close_date'] = pd.to_datetime(events['close_date'], dayfirst=True, errors='coerce')

# 4) Sort & number each event per enterprise
events = (events
          .sort_values(['Tax code number', 'open_date'], ascending=[True, False]))
events['event_idx'] = events.groupby('Tax code number').cumcount()
events['event_idx'] = -events['event_idx']

# 5) Pivot to wide with unstack
df_e = events.set_index(['Tax code number','event_idx'])[['Procedure','open_date','close_date']]
df_un = df_e.unstack('event_idx')  # shape: index=tax codes, columns=(field, idx)
#5.1 order of tuples
field_order = ['Procedure','open_date','close_date']
idx_order   = sorted(df_un.columns.levels[1], reverse=True)

ordered_tuples = [
    (field, idx)
    for field in field_order
    for idx   in idx_order
    if (field, idx) in df_un.columns
]

df_un = df_un.reindex(columns=pd.MultiIndex.from_tuples(ordered_tuples))
# 6) Flatten the MultiIndex columns
df_un.columns = [
    (f"Event {i}"                 if field=='Procedure' else
     f"Date of open Event {i}"   if field=='open_date' else
     f"Date of closure Event {i}")
    for field, i in df_un.columns
]
df_un = df_un.reset_index()  # bring back Tax code number as a column

# 7) Merge back onto master (preserves your enterprise order)
final_df = (master
            .merge(df_un, on='Tax code number', how='left', sort=False)
            .reset_index(drop=True)
           )

#print(f"Enterprises: {len(master)}, rows in final_df: {len(final_df)}")
#final_df.head()

final_df = final_df.drop(columns=[
    'Procedure/cessazione',
    'Date of open procedure/cessazione',
    'Date of closure procedure'
])

In [7]:
#@title Appending
# 1) Treat only "n.a." as missing when reading
read_kwargs = dict(
    sheet_name="Results",
    index_col=None,
    na_values=["n.a."],
    keep_default_na=True
)

# 2) Read & stack all the new data end-to-end
df_list = []
for fn in test_files:
    tmp = pd.read_excel(path_data + fn, **read_kwargs)
    tmp['Tax code number'] = tmp['Tax code number'].astype('Int64')
    df_list.append(tmp)

In [8]:
#@title Balance sheet MANIPULATION - stacking and Fix Duplicate Column Names before mergingh

import re

# 1) Stack all new files into one DataFrame
all_new = pd.concat(df_list, ignore_index=True)


# Function to extract base name (without year) from column name
def get_base_name(col_name):
    """Extract the base variable name without the year"""
    # Pattern to match year at the end (after potential newlines)
    year_pattern = re.compile(r'(20\d{2})$', re.MULTILINE)
    if year_pattern.search(col_name):
        # Remove the year to get base name
        base = year_pattern.sub('', col_name).strip()
        return base
    return None

# Process all_new dataframe to add group prefixes
print("Processing all_new dataframe to fix duplicate column names...")
print(f"Original shape: {all_new.shape}")

# Get all columns
columns = list(all_new.columns)
new_columns = []
group_counter = 1
seen_groups = {}  # Track which base names we've seen and their group numbers
current_base = None
current_group = None

# Process columns in order
for i, col in enumerate(columns):
    base_name = get_base_name(col)

    if base_name is None:
        # Not a year column, keep as is
        new_columns.append(col)
    else:
        # Check if this is a new group
        if base_name != current_base:
            # New group detected
            if base_name in seen_groups:
                # We've seen this base name before, increment group number
                current_group = max(seen_groups[base_name]) + 1
                seen_groups[base_name].append(current_group)
            else:
                # First time seeing this base name
                current_group = group_counter
                seen_groups[base_name] = [current_group]
                group_counter += 1

            current_base = base_name

        # Add prefix to column name
        new_col_name = f"{current_group:04d}_{col}"
        new_columns.append(new_col_name)

# Apply new column names
all_new.columns = new_columns

print(f"\nTotal groups created: {group_counter - 1}")
print(f"Columns renamed: {len([old for old, new in zip(columns, new_columns) if old != new])}")

# Show examples of renamed columns
print("\nExamples of renamed columns:")
year_pattern = re.compile(r'(20\d{2})$', re.MULTILINE)
renamed_examples = [(old, new) for old, new in zip(columns, new_columns)
                    if old != new and year_pattern.search(old)][:10]

for old, new in renamed_examples:
    # Show in readable format
    old_display = old.replace('\n', '\\n')
    new_display = new.replace('\n', '\\n')
    print(f"  {old_display} -> {new_display}")

# Verify no duplicates remain
duplicate_cols = all_new.columns[all_new.columns.duplicated()].tolist()
if duplicate_cols:
    print(f"\n⚠️ Warning: Still have {len(duplicate_cols)} duplicate columns!")
    print(f"Duplicates: {duplicate_cols[:5]}")
else:
    print("\n✅ Success: No duplicate columns remaining!")

# Show column name patterns
print("\nColumn name patterns (first few groups):")
for base_name, groups in list(seen_groups.items())[:5]:
    base_display = base_name.split('\n')[0][:50]  # First line, max 50 chars
    print(f"  '{base_display}...' appears in groups: {groups}")

Processing all_new dataframe to fix duplicate column names...
Original shape: (2000, 4243)

Total groups created: 365
Columns renamed: 3650

Examples of renamed columns:
  Revenues from sales and services\nth EUR\n2023 -> 0001_Revenues from sales and services\nth EUR\n2023
  Revenues from sales and services\nth EUR\n2022 -> 0001_Revenues from sales and services\nth EUR\n2022
  Revenues from sales and services\nth EUR\n2021 -> 0001_Revenues from sales and services\nth EUR\n2021
  Revenues from sales and services\nth EUR\n2020 -> 0001_Revenues from sales and services\nth EUR\n2020
  Revenues from sales and services\nth EUR\n2019 -> 0001_Revenues from sales and services\nth EUR\n2019
  Revenues from sales and services\nth EUR\n2018 -> 0001_Revenues from sales and services\nth EUR\n2018
  Revenues from sales and services\nth EUR\n2017 -> 0001_Revenues from sales and services\nth EUR\n2017
  Revenues from sales and services\nth EUR\n2016 -> 0001_Revenues from sales and services\nth EUR\n201

In [9]:
#@title Merging

# 2) Ensure the key is the same type
all_new['Tax code number'] = all_new['Tax code number'].astype('Int64')

# 3) Perform a 1:1 left-join of your master with the new data,
df = final_df.merge(
    all_new,
    on='Tax code number',
    how='left',
    validate='one_to_one'
)
# Fix duplicate column names if they exist
if 'Company name_x' in df.columns and 'Company name_y' in df.columns:
    # Keep the one from final_df (left side) and drop the duplicate
    df['Company name'] = df['Company name_x']
    df = df.drop(columns=['Company name_x', 'Company name_y'])
    print("Fixed duplicate Company name columns")

print(df.shape)  # rows == final_df.rows; columns == union of both sets of variables


(2000, 4276)


In [10]:
#@title visualize big database

# Show *every* column
pd.set_option('display.max_columns', 1000)

# (Optionally widen your display so lines don’t wrap too crazily)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 50)

# Now when you do…
#df

#print(final_df.iloc[:, :10])
#df.iloc[:30, :200]
#df.iloc[1000:1030, :200]

df.iloc[985:1015, :200]

,Unnamed: 0_x,Company name_x,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Date of closure Event -6,Unnamed: 0_y,Company name_y,0001_Revenues from sales and services\nth EUR\n2023,0001_Revenues from sales and services\nth EUR\n2022,0001_Revenues from sales and services\nth EUR\n2021,0001_Revenues from sales and services\nth EUR\n2020,0001_Revenues from sales and services\nth EUR\n2019,0001_Revenues from sales and services\nth EUR\n2018,0001_Revenues from sales and services\nth EUR\n2017,0001_Revenues from sales and services\nth EUR\n2016,0001_Revenues from sales and services\nth EUR\n2015,0001_Revenues from sales and services\nth EUR\n2014,0002_EBITDA\nth EUR\n2023,0002_EBITDA\nth EUR\n2022,0002_EBITDA\nth EUR\n2021,0002_EBITDA\nth EUR\n2020,0002_EBITDA\nth EUR\n2019,0002_EBITDA\nth EUR\n2018,0002_EBITDA\nth EUR\n2017,0002_EBITDA\nth EUR\n2016,0002_EBITDA\nth EUR\n2015,0002_EBITDA\nth EUR\n2014,0003_Profit (loss)\nth EUR\n2023,0003_Profit (loss)\nth EUR\n2022,0003_Profit (loss)\nth EUR\n2021,0003_Profit (loss)\nth EUR\n2020,0003_Profit (loss)\nth EUR\n2019,0003_Profit (loss)\nth EUR\n2018,0003_Profit (loss)\nth EUR\n2017,0003_Profit (loss)\nth EUR\n2016,0003_Profit (loss)\nth EUR\n2015,0003_Profit (loss)\nth EUR\n2014,0004_Total assets\nth EUR\n2023,0004_Total assets\nth EUR\n2022,0004_Total assets\nth EUR\n2021,0004_Total assets\nth EUR\n2020,0004_Total assets\nth EUR\n2019,0004_Total assets\nth EUR\n2018,0004_Total assets\nth EUR\n2017,0004_Total assets\nth EUR\n2016,0004_Total assets\nth EUR\n2015,0004_Total assets\nth EUR\n2014,0005_Total shareholder's funds\nth EUR\n2023,0005_Total shareholder's funds\nth EUR\n2022,0005_Total shareholder's funds\nth EUR\n2021,0005_Total shareholder's funds\nth EUR\n2020,0005_Total shareholder's funds\nth EUR\n2019,0005_Total shareholder's funds\nth EUR\n2018,0005_Total shareholder's funds\nth EUR\n2017,0005_Total shareholder's funds\nth EUR\n2016,0005_Total shareholder's funds\nth EUR\n2015,0005_Total shareholder's funds\nth EUR\n2014,0006_Net financial position\nth EUR\n2023,0006_Net financial position\nth EUR\n2022,0006_Net financial position\nth EUR\n2021,0006_Net financial position\nth EUR\n2020,0006_Net financial position\nth EUR\n2019,0006_Net financial position\nth EUR\n2018,0006_Net financial position\nth EUR\n2017,0006_Net financial position\nth EUR\n2016,0006_Net financial position\nth EUR\n2015,0006_Net financial position\nth EUR\n2014,0007_EBITDA/Vendite\n%\n2023,0007_EBITDA/Vendite\n%\n2022,0007_EBITDA/Vendite\n%\n2021,0007_EBITDA/Vendite\n%\n2020,0007_EBITDA/Vendite\n%\n2019,0007_EBITDA/Vendite\n%\n2018,0007_EBITDA/Vendite\n%\n2017,0007_EBITDA/Vendite\n%\n2016,0007_EBITDA/Vendite\n%\n2015,0007_EBITDA/Vendite\n%\n2014,0008_Return on sales (ROS)\n%\n2023,0008_Return on sales (ROS)\n%\n2022,0008_Return on sales (ROS)\n%\n2021,0008_Return on sales (ROS)\n%\n2020,0008_Return on sales (ROS)\n%\n2019,0008_Return on sales (ROS)\n%\n2018,0008_Return on sales (ROS)\n%\n2017,0008_Return on sales (ROS)\n%\n2016,0008_Return on sales (ROS)\n%\n2015,0008_Return on sales (ROS)\n%\n2014,0009_Return on asset (ROA)\n%\n2023,0009_Return on asset (ROA)\n%\n2022,0009_Return on asset (ROA)\n%\n2021,0009_Return on asset (ROA)\n%\n2020,0009_Return on asset (ROA)\n%\n2019,0009_Return on asset (ROA)\n%\n2018,0009_Return on asset (ROA)\n%\n2017,0009_Return on asset (ROA)\n%\n2016,0009_Return on asset (ROA)\n%\n2015,0009_Ret

In [11]:
print(df.shape)

(2000, 4276)


In [12]:
#@title LIST MISSING VARIABLES

# 1) Count columns that are 100% missing
all_nan = df.columns[df.isna().all()]
print(f"Columns with all NaN: {len(all_nan)}")

# 2) Drop them
df.drop(columns=all_nan, inplace=True)

# 3) Show the list of dropped columns
print("Dropped columns:")
for col in all_nan:
    print(f" - {col}")


Columns with all NaN: 759
Dropped columns:
 - Date of closure Event -6
 - 0081_Amounts due for advance taxation
th EUR
2023
 - 0081_Amounts due for advance taxation
th EUR
2022
 - 0081_Amounts due for advance taxation
th EUR
2021
 - 0081_Amounts due for advance taxation
th EUR
2020
 - 0081_Amounts due for advance taxation
th EUR
2019
 - 0081_Amounts due for advance taxation
th EUR
2018
 - 0081_Amounts due for advance taxation
th EUR
2017
 - 0081_Amounts due for advance taxation
th EUR
2016
 - 0081_Amounts due for advance taxation
th EUR
2015
 - 0081_Amounts due for advance taxation
th EUR
2014
 - 0109_GROUP consolidation reserve
th EUR
2023
 - 0109_GROUP consolidation reserve
th EUR
2022
 - 0109_GROUP consolidation reserve
th EUR
2021
 - 0109_GROUP consolidation reserve
th EUR
2020
 - 0109_GROUP consolidation reserve
th EUR
2019
 - 0109_GROUP consolidation reserve
th EUR
2018
 - 0109_GROUP consolidation reserve
th EUR
2017
 - 0109_GROUP consolidation reserve
th EUR
2016
 - 0109_GROUP c

In [13]:
#@title RANK OF MISSING in NON-MISSING VARIABLES

# 4) Compute missing counts for the rest, sort descending
missing_counts = df.isna().sum().sort_values(ascending=False)

# Ensure pandas will print all rows
pd.set_option('display.max_rows', None)

print("\nMissing‐value counts for remaining columns (desc):")
print(missing_counts)



Missing‐value counts for remaining columns (desc):
0257_Cash flow from investments (B)\nth EUR\n2015                                                                                          1999
0256_Disposal of business and interests in affiliates, net of cash acquired\nth EUR\n2020                                                  1999
0256_Disposal of business and interests in affiliates, net of cash acquired\nth EUR\n2022                                                  1999
0255_Acquisition of business and interests in affiliates, net of cash acquired\nth EUR\n2019                                               1999
0255_Acquisition of business and interests in affiliates, net of cash acquired\nth EUR\n2018                                               1999
0255_Acquisition of business and interests in affiliates, net of cash acquired\nth EUR\n2020                                               1999
0255_Acquisition of business and interests in affiliates, net of cash acquired\nth E

In [14]:
#@title SAVE PROCESSED DATASET (Wide Format)
import pickle

# Save the processed wide format dataset
save_path = path_data + 'processed_df_wide.pkl'
df.to_pickle(save_path)
print(f"✅ Saved processed dataset to: {save_path}")
print(f"Shape: {df.shape}, Size: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

✅ Saved processed dataset to: /content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/model_test/processed_df_wide.pkl
Shape: (2000, 3517), Size: 60.4 MB


### (#) open dataset created

In [ ]:
#@title LOAD PROCESSED DATASET (Wide Format)
import pickle
import pandas as pd
import re

# Load the pre-processed dataset
load_path = path_data + 'processed_df_wide.pkl'

try:
    df = pd.read_pickle(load_path)
    print(f"✅ Loaded processed dataset from: {load_path}")
    print(f"Shape: {df.shape}")
    print(f"Columns with 'date'/'year': {len([c for c in df.columns if re.search(r'(year|date)$', c, re.IGNORECASE)])}")

    # Quick validation that dates are still datetime objects
    date_cols = [c for c in df.columns if re.search(r'(year|date)$', c, re.IGNORECASE)]
    if date_cols:
        sample_col = date_cols[0]
        print(f"Date dtype check - {sample_col}: {df[sample_col].dtype}")

    # Check event columns
    event_cols = [c for c in df.columns if c.startswith('Event ')]
    print(f"Event columns found: {len(event_cols)}")

except FileNotFoundError:
    print("❌ Processed file not found. Run the full import process first.")
except Exception as e:
    print(f"❌ Error loading processed file: {e}")

✅ Loaded processed dataset from: /content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/model_test/processed_df_wide.pkl
Shape: (2000, 3517)
Columns with 'date'/'year': 2
Date dtype check - Incorporation year: datetime64[ns]
Event columns found: 7


### stats

---



In [ ]:
#@title missing variables
# 1) Row‐count sanity: you should still have exactly one row per tax code
n_rows = df.shape[0]
n_unique_codes = df['Tax code number'].nunique()
print(f"Rows: {n_rows}, Unique Tax codes: {n_unique_codes}")

# 2) Check for any duplicate tax codes (should be zero)
dups = df['Tax code number'].duplicated().sum()
print(f"Duplicate keys: {dups}")

# 3) Overall missing‐value summary
missing_per_col = df.isna().sum().sort_values(ascending=False)
print("Top 10 columns by # missing values:")
print(missing_per_col.iloc[30:90])

Rows: 2000, Unique Tax codes: 2000
Duplicate keys: 0
Top 10 columns by # missing values:
Tax code number                            0
Legal status                               0
Company category                           0
Registered office address - Postal code    0
dtype: int64


In [ ]:
#@title only missing variables

# 1) Compute total missing per column
missing = df.isna().sum()

# 2) Filter to only those with at least one missing
missing = missing[missing > 0]

# 3) Sort descending (largest gaps first)
missing = missing.sort_values(ascending=False)

# 4) Make sure pandas will print every row
pd.set_option('display.max_rows', None)

# 5) Now display
print(missing)


Total cash and cash equivalents at year end\n2018    2000
Total cash and cash equivalents at year end\n2020    2000
Total cash and cash equivalents at year end\n2019    2000
Total cash and cash equivalents at year end\n2021    2000
Total cash and cash equivalents at year end\n2022    2000
Total cash and cash equivalents at year end\n2023    2000
Date of closure Event -6                             2000
Total cash and cash equivalents at year end\n2017    2000
Total cash and cash equivalents at year end\n2014    2000
Total cash and cash equivalents at year end\n2016    2000
Total cash and cash equivalents at year end\n2015    2000
Date of closure Event -5                             1998
Event -6                                             1998
Date of open Event -6                                1998
Date of closure Event -4                             1997
Event -5                                             1995
Date of open Event -5                                1995
Date of closur

In [ ]:
#@title visualize big part of database

from google.colab import data_table

# Turn on Colab’s interactive DataFrame display
data_table.enable_dataframe_formatter()

# Now this will show you an interactive, scrollable table
# (you can still slice it to a column range if you like)
#display(df)

# — or to only see columns 20 through 39 —
display(df.iloc[:, 170:200])


In [ ]:
#@title CHECK COLLINEARITY, MISSINGNESS & MORE (Corrected)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1) Missing‐value summary on df
miss_pct = df.isnull().mean().sort_values(ascending=False)
print(">> Top 10 columns by percent missing:")
print(miss_pct.head(100).apply(lambda x: f"{x:.1%}"))
print("\n>> Columns with 100% missing:")
print(miss_pct[miss_pct == 1.0].index.tolist())

# 2) Data‐type & cardinality overview (drop the list‐column if present)
if 'ALL_events' in df.columns:
    basic = df.drop(columns=['ALL_events'])
else:
    basic = df.copy()

nunique = basic.nunique(dropna=False)
dtype_df = pd.DataFrame({
    'dtype':    basic.dtypes.astype(str),
    'n_unique': nunique
})
print("\n>> Data types and unique counts (sorted by dtype):")
print(dtype_df.sort_values('n_unique').head(10))

# 3) Correlation on numeric features (excluding fully‐missing columns)
#    Select columns with <100% missing, then pick numeric
valid_cols = miss_pct[miss_pct < 1.0].index
num = df[valid_cols].select_dtypes(include=[np.number])

corr = num.corr().abs()
high_corr = (
    corr.where(np.tril(np.ones(corr.shape), k=-1).astype(bool))
        .stack()
        .loc[lambda s: s > 0.90]
        .sort_values(ascending=False)
)
print(f"\n>> Highly correlated pairs (>|0.90|) [count={len(high_corr)}]:")
print(high_corr.head(10))

# 4) Variance Inflation Factor (VIF)
num_for_vif = num.fillna(0).astype(float)
vif_df = pd.DataFrame({
    'variable': num_for_vif.columns,
    'VIF': [
        variance_inflation_factor(num_for_vif.values, i)
        for i in range(num_for_vif.shape[1])
    ]
}).sort_values('VIF', ascending=False)
print("\n>> Top 10 variables by VIF:")
print(vif_df.head(10))

# 5) Distribution diagnostics for numeric columns
print("\n>> Skewness of numeric features:")
print(num.skew().sort_values(ascending=False).head(10))

outliers = ((num - num.mean()).abs() > 3 * num.std()) \
            .sum() \
            .sort_values(ascending=False)
print("\n>> Count of outliers (>|3σ|) per numeric feature:")
print(outliers.head(10))


### TEST COLAB

---



In [ ]:
#@title Bankruptcy dummy
import pandas as pd
import numpy as np

# Create bankruptcy dummy and date variables
df['Bankruptcy_dummy'] = 0
df['Bankruptcy_dummy_date'] = pd.NaT

# Find all event columns
event_cols = [c for c in df.columns if c.startswith('Event ') and not c.startswith('Date of')]

print(f"Checking {len(event_cols)} event columns for bankruptcy...")

# For each row, check if any event contains "Bankruptcy"
bankruptcy_count = 0

for idx, row in df.iterrows():
    for event_col in event_cols:
        event_value = row[event_col]

        # Check if this event is exactly "Bankruptcy" (case sensitive)
        if pd.notna(event_value) and event_value == 'Bankruptcy':
            # Set the dummy to 1
            df.loc[idx, 'Bankruptcy_dummy'] = 1

            # Extract the event number from column name (e.g., "Event -2" -> "-2")
            event_num = event_col.replace('Event ', '')

            # Find the corresponding date column
            date_col = f'Date of open Event {event_num}'

            # If the date column exists and has a value, store it
            if date_col in df.columns and pd.notna(row[date_col]):
                df.loc[idx, 'Bankruptcy_dummy_date'] = pd.to_datetime(row[date_col])
                bankruptcy_count += 1
                break  # Stop after finding the first bankruptcy event

# Convert date column to datetime type
df['Bankruptcy_dummy_date'] = pd.to_datetime(df['Bankruptcy_dummy_date'])

# Display summary
print(f"\nTotal enterprises with Bankruptcy event: {df['Bankruptcy_dummy'].sum()}")
print(f"Enterprises with valid bankruptcy dates: {bankruptcy_count}")

# Show distribution
print("\nBankruptcy_dummy distribution:")
print(df['Bankruptcy_dummy'].value_counts())

# Show sample of enterprises with bankruptcy
bankrupt_sample = df[df['Bankruptcy_dummy'] == 1][['Tax code number', 'Company name', 'Bankruptcy_dummy', 'Bankruptcy_dummy_date']].head(10)
print("\nSample of enterprises with bankruptcy:")
display(bankrupt_sample)

Checking 7 event columns for bankruptcy...

Total enterprises with Bankruptcy event: 301
Enterprises with valid bankruptcy dates: 300

Bankruptcy_dummy distribution:
Bankruptcy_dummy
0    1699
1     301
Name: count, dtype: int64

Sample of enterprises with bankruptcy:


,Tax code number,Company name_x,Bankruptcy_dummy,Bankruptcy_dummy_date
8,647460286,VIMET S.P.A.,1,2017-03-03
36,4261290821,FENICE DISTRIBUZIONE S.R.L.,1,2017-05-15
53,2229420415,BENELLI Q. J. SRL,1,2016-07-12
67,1614500799,LA NUOVA NAVE SOCIETA' A RESPONSABILITA' LIMIT...,1,2021-10-12
71,3074840160,ONGIS METAL FER S.P.A.,1,2018-03-16
76,301260725,ALBERGO PETROLI SOCIETA' A RESPONSABILITA' LIM...,1,2021-11-08
94,3725790137,NUOVA REX SUPERMERCATI S.R.L.,1,2021-07-22
97,1697130464,PERINI NAVI SOCIETA' PER AZIONI,1,2021-01-29
100,6850091007,JENNIFER PETROLI S.R.L. IN LIQUIDAZIONE,1,2017-01-10
102,13116741003,CARFUEL S.R.L.,1,2018-05-08


In [ ]:
#@title Reshape Long
import pandas as pd
import numpy as np
import re
import gc

# Starting from `df` which has Bankruptcy_dummy and Bankruptcy_dummy_date

# 1. First, identify columns with years
year_pattern = re.compile(r'(20\d{2})$', re.MULTILINE)
wide_cols = [col for col in df.columns if year_pattern.search(col)]
id_cols = [c for c in df.columns if c not in wide_cols]

print(f"Found {len(wide_cols)} columns with years")
print(f"Number of ID columns: {len(id_cols)}")
print(f"Total enterprises: {len(df)}")

# 2. Create a function to extract variable name and year
def parse_column_name(col_name):
    """Extract clean variable name and year from column"""
    year_match = year_pattern.search(col_name)
    if year_match:
        year = int(year_match.group(1))
        # Get variable name (first line before \n)
        var_name = col_name.split('\n')[0].strip()
        return var_name, year
    return None, None

# 3. Get unique years and variables
years = set()
variables = set()
col_mapping = {}  # Maps (variable, year) to original column name

for col in wide_cols:
    var_name, year = parse_column_name(col)
    if var_name and year:
        years.add(year)
        variables.add(var_name)
        col_mapping[(var_name, year)] = col

years = sorted(years)
variables = sorted(variables)

print(f"\nYears found: {min(years)} to {max(years)} (Total: {len(years)} years)")
print(f"Number of unique variables: {len(variables)}")
print(f"Expected max rows (if no bankruptcy): {len(df) * len(years):,}")

# 4. Process in chunks to save memory
chunk_size = 100  # Process 100 enterprises at a time
long_chunks = []
total_rows_created = 0
enterprises_truncated = 0

print("\nProcessing in chunks to save memory...")

for chunk_start in range(0, len(df), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(df))

    # Get chunk of enterprises
    df_chunk = df.iloc[chunk_start:chunk_end]

    # Create list to store long format data for this chunk
    chunk_data = []

    for _, row in df_chunk.iterrows():
        tax_code = row['Tax code number']

        # Get bankruptcy year if exists
        bankruptcy_year = None
        if row['Bankruptcy_dummy'] == 1 and pd.notna(row['Bankruptcy_dummy_date']):
            bankruptcy_year = row['Bankruptcy_dummy_date'].year
            enterprises_truncated += 1

        # For each year
        rows_for_enterprise = 0
        for year in years:
            # Skip years after bankruptcy
            if bankruptcy_year and year > bankruptcy_year:
                continue

            # Create row for this enterprise-year
            long_row = {}

            # Add ID columns (including Bankruptcy_dummy and Bankruptcy_dummy_date)
            for id_col in id_cols:
                long_row[id_col] = row[id_col]

            # Add year
            long_row['Year'] = year

            # Add variable values for this year
            for var_name in variables:
                col_key = (var_name, year)
                if col_key in col_mapping:
                    orig_col = col_mapping[col_key]
                    if orig_col in row.index:
                        long_row[var_name] = row[orig_col]
                    else:
                        long_row[var_name] = np.nan
                else:
                    long_row[var_name] = np.nan

            chunk_data.append(long_row)
            rows_for_enterprise += 1
            total_rows_created += 1

    # Convert chunk to dataframe
    chunk_long = pd.DataFrame(chunk_data)
    long_chunks.append(chunk_long)

    # Progress update
    if (chunk_end % 500 == 0) or (chunk_end == len(df)):
        print(f"  Processed {chunk_end}/{len(df)} enterprises... Total rows created: {total_rows_created:,}")

    # Clean up memory
    del chunk_data
    gc.collect()

# 5. Combine all chunks
print("\nCombining chunks...")
long_df = pd.concat(long_chunks, ignore_index=True)

# Clean up
del long_chunks
gc.collect()

# 6. Add derived columns
print("\nAdding derived columns...")

# Create NameYear using the actual Company name column
if 'Company name' in long_df.columns:
    long_df['NameYear'] = long_df['Company name'].astype(str) + ' ' + long_df['Year'].astype(str)
    print("Using 'Company name' column for NameYear")
else:
    print("Warning: 'Company name' column not found!")

# Create KeyYear using Tax code number + Year
long_df['KeyYear'] = long_df['Tax code number'].astype(str) + '_' + long_df['Year'].astype(str)

# 7. Sort the data
long_df = long_df.sort_values(['Tax code number', 'Year']).reset_index(drop=True)

# 8. Display summary
print(f"\nFinal shape: {long_df.shape}")
print(f"Memory usage: {long_df.memory_usage(deep=True).sum() / 1024**3:.2f} GB")
print(f"Unique enterprises: {long_df['Tax code number'].nunique()}")
print(f"Years covered: {long_df['Year'].min()} to {long_df['Year'].max()}")
print(f"Enterprises truncated due to bankruptcy: {enterprises_truncated}")

# Check observation count per enterprise
obs_per_enterprise = long_df.groupby('Tax code number').size()
print(f"\nObservations per enterprise:")
print(obs_per_enterprise.value_counts().sort_index())

# 9. Save to file
print("\nSaving to pickle file...")
long_df.to_pickle(path_data + 'long_format_data.pkl')
print("Saved successfully!")

# 10. Show sample
print("\nSample of reshaped data (first 20 rows):")
display(long_df.head(20))

# Show a company with bankruptcy
bankrupt_companies = long_df[long_df['Bankruptcy_dummy'] == 1]['Tax code number'].unique()
if len(bankrupt_companies) > 0:
    sample_bankrupt = bankrupt_companies[0]
    print(f"\nSample bankrupt company {sample_bankrupt}:")
    display(long_df[long_df['Tax code number'] == sample_bankrupt])

# Show columns
print("\nColumns in long format (first 50):")
for i, col in enumerate(long_df.columns[:50]):
    print(f"  {i+1}. {col}")

Found 3290 columns with years
Number of ID columns: 229
Total enterprises: 2000

Years found: 2014 to 2023 (Total: 10 years)
Number of unique variables: 338
Expected max rows (if no bankruptcy): 20,000

Processing in chunks to save memory...
  Processed 500/2000 enterprises... Total rows created: 4,545
  Processed 1000/2000 enterprises... Total rows created: 8,761
  Processed 1500/2000 enterprises... Total rows created: 13,761
  Processed 2000/2000 enterprises... Total rows created: 18,761

Combining chunks...

Adding derived columns...

Final shape: (18761, 569)
Memory usage: 0.10 GB
Unique enterprises: 1995
Years covered: 2014 to 2023
Enterprises truncated due to bankruptcy: 300

Observations per enterprise:
1        2
3        9
4       49
5       64
6       67
7       38
8       47
9       18
10    1701
Name: count, dtype: int64

Saving to pickle file...
Saved successfully!

Sample of reshaped data (first 20 rows):


,Unnamed: 0_x,Company name_x,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Unnamed: 0_y,Company name_y,Additions in progress and advances\nth EUR\n2023.1,Additions in progress and advances\nth EUR\n2022.1,Additions in progress and advances\nth EUR\n2021.1,Additions in progress and advances\nth EUR\n2020.1,Additions in progress and advances\nth EUR\n2019.1,Additions in progress and advances\nth EUR\n2018.1,Additions in progress and advances\nth EUR\n2017.1,Additions in progress and advances\nth EUR\n2016.1,Additions in progress and advances\nth EUR\n2015.1,Additions in progress and advances\nth EUR\n2014.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2023.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2022.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2021.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2020.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2019.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2018.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2017.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2016.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2015.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2014.1,Receiv. due from subs. Comp.\nth EUR\n2023.1,Receiv. due from subs. Comp.\nth EUR\n2022.1,Receiv. due from subs. Comp.\nth EUR\n2021.1,Receiv. due from subs. Comp.\nth EUR\n2020.1,Receiv. due from subs. Comp.\nth EUR\n2019.1,Receiv. due from subs. Comp.\nth EUR\n2018.1,Receiv. due from subs. Comp.\nth EUR\n2017.1,Receiv. due from subs. Comp.\nth EUR\n2016.1,Receiv. due from subs. Comp.\nth EUR\n2015.1,Receiv. due from subs. Comp.\nth EUR\n2014.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2023.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2022.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2021.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2020.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2019.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2018.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2017.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2016.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2015.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2014.2,Due from comp. under parent companies control\nth EUR\n2023.1,Due from comp. under parent companies control\nth EUR\n2022.1,Due from comp. under parent companies control\nth EUR\n2021.1,Due from comp. under parent companies control\nth EUR\n2020.1,Due from comp. under parent companies control\nth EUR\n2019.1,Due from comp. under parent companies control\nth EUR\n2018.1,Due from comp. under parent companies control\nth EUR\n2017.1,Due from comp. under parent companies control\nth EUR\n2016.1,Due from comp. under parent companies control\nth EUR\n2015.1,Due from comp. under parent companies control\nth EUR\n2014.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2023.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2022.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2021.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2020.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2019.1,Due from comp. under p


Sample bankrupt company 124550401:


,Unnamed: 0_x,Company name_x,Tax code number,Registered office address - Postal code,Registered office address - Longitude,Registered office address - Latitude,Trading address - Postal code,Legal status,Legal form,Company category,Incorporation year,No of available years,Last accounting closing date,Event 0,Event -1,Event -2,Event -3,Event -4,Event -5,Event -6,Date of open Event 0,Date of open Event -1,Date of open Event -2,Date of open Event -3,Date of open Event -4,Date of open Event -5,Date of open Event -6,Date of closure Event 0,Date of closure Event -1,Date of closure Event -2,Date of closure Event -3,Date of closure Event -4,Date of closure Event -5,Unnamed: 0_y,Company name_y,Additions in progress and advances\nth EUR\n2023.1,Additions in progress and advances\nth EUR\n2022.1,Additions in progress and advances\nth EUR\n2021.1,Additions in progress and advances\nth EUR\n2020.1,Additions in progress and advances\nth EUR\n2019.1,Additions in progress and advances\nth EUR\n2018.1,Additions in progress and advances\nth EUR\n2017.1,Additions in progress and advances\nth EUR\n2016.1,Additions in progress and advances\nth EUR\n2015.1,Additions in progress and advances\nth EUR\n2014.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2023.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2022.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2021.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2020.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2019.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2018.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2017.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2016.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2015.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2014.1,Receiv. due from subs. Comp.\nth EUR\n2023.1,Receiv. due from subs. Comp.\nth EUR\n2022.1,Receiv. due from subs. Comp.\nth EUR\n2021.1,Receiv. due from subs. Comp.\nth EUR\n2020.1,Receiv. due from subs. Comp.\nth EUR\n2019.1,Receiv. due from subs. Comp.\nth EUR\n2018.1,Receiv. due from subs. Comp.\nth EUR\n2017.1,Receiv. due from subs. Comp.\nth EUR\n2016.1,Receiv. due from subs. Comp.\nth EUR\n2015.1,Receiv. due from subs. Comp.\nth EUR\n2014.1,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2023.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2022.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2021.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2020.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2019.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2018.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2017.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2016.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2015.2,Receiv. due from parent comp. - beyond 12 months\nth EUR\n2014.2,Due from comp. under parent companies control\nth EUR\n2023.1,Due from comp. under parent companies control\nth EUR\n2022.1,Due from comp. under parent companies control\nth EUR\n2021.1,Due from comp. under parent companies control\nth EUR\n2020.1,Due from comp. under parent companies control\nth EUR\n2019.1,Due from comp. under parent companies control\nth EUR\n2018.1,Due from comp. under parent companies control\nth EUR\n2017.1,Due from comp. under parent companies control\nth EUR\n2016.1,Due from comp. under parent companies control\nth EUR\n2015.1,Due from comp. under parent companies control\nth EUR\n2014.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2023.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2022.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2021.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2020.1,Due from comp. under parent companies control - beyond 12 months\nth EUR\n2019.1,Due from comp. under p


Columns in long format (first 50):
  1. Unnamed: 0_x
  2. Company name_x
  3. Tax code number
  4. Registered office address - Postal code
  5. Registered office address - Longitude
  6. Registered office address - Latitude
  7. Trading address - Postal code
  8. Legal status
  9. Legal form
  10. Company category
  11. Incorporation year
  12. No of available years
  13. Last accounting closing date
  14. Event 0
  15. Event -1
  16. Event -2
  17. Event -3
  18. Event -4
  19. Event -5
  20. Event -6
  21. Date of open Event 0
  22. Date of open Event -1
  23. Date of open Event -2
  24. Date of open Event -3
  25. Date of open Event -4
  26. Date of open Event -5
  27. Date of open Event -6
  28. Date of closure Event 0
  29. Date of closure Event -1
  30. Date of closure Event -2
  31. Date of closure Event -3
  32. Date of closure Event -4
  33. Date of closure Event -5
  34. Unnamed: 0_y
  35. Company name_y
  36. Additions in progress and advances
th EUR
2023.1
  37. Additions 

In [ ]:
#long_df.head()
#df.head()
long_df.iloc[985:1015, :200]
#df.iloc[985:1015, :200]

In [ ]:
#@title SAVE LONG FORMAT DATASET
# This is already in your reshape code, but here's a standalone version
if 'long_df' in locals():
    save_path_long = path_data + 'long_format_data.pkl'
    long_df.to_pickle(save_path_long)
    print(f"✅ Saved long format dataset to: {save_path_long}")
    print(f"Shape: {long_df.shape}, Size: {long_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
else:
    print("❌ long_df not found. Run the Reshape Long cell first.")

✅ Saved long format dataset to: /content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/model_test/long_format_data.pkl
Shape: (18761, 569), Size: 99.9 MB


In [ ]:
#@title LOAD LONG FORMAT DATASET
import pickle
import pandas as pd

load_path_long = path_data + 'long_format_data.pkl'

try:
    long_df = pd.read_pickle(load_path_long)
    print(f"✅ Loaded long format dataset from: {load_path_long}")
    print(f"Shape: {long_df.shape}")
    print(f"Unique enterprises: {long_df['Tax code number'].nunique()}")
    print(f"Years covered: {long_df['Year'].min()} to {long_df['Year'].max()}")

    # Check key variables
    if 'KeyYear' in long_df.columns:
        print("✅ KeyYear variable present")
    if 'NameYear' in long_df.columns:
        print("✅ NameYear variable present")

    # Show observation distribution
    obs_per_enterprise = long_df.groupby('Tax code number').size()
    print(f"\nObservations per enterprise distribution:")
    print(obs_per_enterprise.value_counts().sort_index().head())

except FileNotFoundError:
    print("❌ Long format file not found. Run the Reshape Long process first.")
except Exception as e:
    print(f"❌ Error loading long format file: {e}")

✅ Loaded long format dataset from: /content/drive/MyDrive/SETUP DEFINITIVO/Temporaneo, lezioni ML/AIDA/DATA/model_test/long_format_data.pkl
Shape: (18761, 569)
Unique enterprises: 1995
Years covered: 2014 to 2023
✅ KeyYear variable present

Observations per enterprise distribution:
1     2
3     9
4    49
5    64
6    67
Name: count, dtype: int64


In [ ]:
df.iloc[985:1015, 1000:1200]

,Derivatives\nth EUR\n2018.1,Derivatives\nth EUR\n2017.1,Derivatives\nth EUR\n2016.1,Derivatives\nth EUR\n2015.1,Derivatives\nth EUR\n2014.1,Other securities\nth EUR\n2023.1,Other securities\nth EUR\n2022.1,Other securities\nth EUR\n2021.1,Other securities\nth EUR\n2020.1,Other securities\nth EUR\n2019.1,Other securities\nth EUR\n2018.1,Other securities\nth EUR\n2017.1,Other securities\nth EUR\n2016.1,Other securities\nth EUR\n2015.1,Other securities\nth EUR\n2014.1,0089_Financial instruments for cash pooling\nth EUR\n2023,0089_Financial instruments for cash pooling\nth EUR\n2022,0089_Financial instruments for cash pooling\nth EUR\n2021,0089_Financial instruments for cash pooling\nth EUR\n2020,0089_Financial instruments for cash pooling\nth EUR\n2019,0089_Financial instruments for cash pooling\nth EUR\n2018,0089_Financial instruments for cash pooling\nth EUR\n2017,0089_Financial instruments for cash pooling\nth EUR\n2016,0089_Financial instruments for cash pooling\nth EUR\n2015,0089_Financial instruments for cash pooling\nth EUR\n2014,0090_TOTAL LIQUID FUNDS\nth EUR\n2023,0090_TOTAL LIQUID FUNDS\nth EUR\n2022,0090_TOTAL LIQUID FUNDS\nth EUR\n2021,0090_TOTAL LIQUID FUNDS\nth EUR\n2020,0090_TOTAL LIQUID FUNDS\nth EUR\n2019,0090_TOTAL LIQUID FUNDS\nth EUR\n2018,0090_TOTAL LIQUID FUNDS\nth EUR\n2017,0090_TOTAL LIQUID FUNDS\nth EUR\n2016,0090_TOTAL LIQUID FUNDS\nth EUR\n2015,0090_TOTAL LIQUID FUNDS\nth EUR\n2014,0091_Bank and postal deposits\nth EUR\n2023,0091_Bank and postal deposits\nth EUR\n2022,0091_Bank and postal deposits\nth EUR\n2021,0091_Bank and postal deposits\nth EUR\n2020,0091_Bank and postal deposits\nth EUR\n2019,0091_Bank and postal deposits\nth EUR\n2018,0091_Bank and postal deposits\nth EUR\n2017,0091_Bank and postal deposits\nth EUR\n2016,0091_Bank and postal deposits\nth EUR\n2015,0091_Bank and postal deposits\nth EUR\n2014,0092_Checks\nth EUR\n2023,0092_Checks\nth EUR\n2022,0092_Checks\nth EUR\n2021,0092_Checks\nth EUR\n2020,0092_Checks\nth EUR\n2019,0092_Checks\nth EUR\n2018,0092_Checks\nth EUR\n2017,0092_Checks\nth EUR\n2016,0092_Checks\nth EUR\n2015,0092_Checks\nth EUR\n2014,0093_Cash and cash equivalents\nth EUR\n2023,0093_Cash and cash equivalents\nth EUR\n2022,0093_Cash and cash equivalents\nth EUR\n2021,0093_Cash and cash equivalents\nth EUR\n2020,0093_Cash and cash equivalents\nth EUR\n2019,0093_Cash and cash equivalents\nth EUR\n2018,0093_Cash and cash equivalents\nth EUR\n2017,0093_Cash and cash equivalents\nth EUR\n2016,0093_Cash and cash equivalents\nth EUR\n2015,0093_Cash and cash equivalents\nth EUR\n2014,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2023,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2022,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2021,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2020,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2019,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2018,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2017,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2016,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2015,0094_TOTAL ACCRUED INCOME AND PREPAID EXPENSES\nth EUR\n2014,0095_Accrued income and prepaid exp.\nth EUR\n2023,0095_Accrued income and prepaid exp.\nth EUR\n2022,0095_Accrued income and prepaid exp.\nth EUR\n2021,0095_Accrued income and prepaid exp.\nth EUR\n2020,0095_Accrued income and prepaid exp.\nth EUR\n2019,0095_Accrued income and prepaid exp.\nth EUR\n2018,0095_Accrued income and prepaid exp.\nth EUR\n2017,0095_Accrued income and prepaid exp.\nth EUR\n2016,0095_Accrued income and prepaid exp.\nth EUR\n2015,0095_Accrued income and prepaid exp.\nth EUR\n2014,0096_TOTAL ASSETS\nth EUR\n2023,0096_TOTAL ASSETS\nth EUR\n2022,0096_TOTAL ASSETS\nth EUR\n2021,0096_TOTAL ASSETS\nth EUR\n2020,0096_TOTAL ASSETS\nth EUR\n2019,0096_TOTAL ASSETS\nth EUR\n2018,0096_TOTAL ASSETS\nth EUR\n2017,0096_TOTAL ASSETS\nth EUR\n2016,0096_TOTAL ASSETS\nth EUR\n2015,0096_TOTAL 